### 손글씨 이미지 데이터 MNIST

- 인코딩된 바이너리 데이터를 디코딩하여 처리하는 방식 확인
- 지도 학습
- 학습용 데이터는 6만개, 테스트 데이터는 1만개
- 결론
    - 학습 후 새로운 데이터 입력시 판별
    - 0~9까지의 손글씨 이미지를 판별
    - 데이터는 url을 직접 획득해서, 원하는 곳에 다운로드 시키겠다
    

### 절차

|No|단계|내용|
|:---:|:---|:---|
|1|연구목표|- 손글씨 이미지(0-9)를 학습시켜서, 새로운 손글씨 이미지를 판별해 내는 머신러닝 모델을 구축<br>- 압축된 이미지를 압축해제<br>- 인코딩된 데이터를 디코딩 처리<br>- 28X28로 구성된 픽셀 이미지 데이터를 벡터화 처리<br>- 시스템 통합의 결과를 복 연구 목표를 설정해야 하지만 시스템 통합을 생략하므로 이부분은 생략|
|2|데이터획득/수집|- http://yann.lecun.com/exdb/mnist/ 접속<br>- web scraping을 통해서 데이터의 URL 획득<br>- 지정된 위치에 다운로드->압축해제|
|3|데이터준비/전처리|- 디코딩(내부구조를 알수 있는 인코딩 문서(MNIST Database) 필요)<br>- 에디언(Endian)처리<br>- 벡터화 처리|
|4|데이터탐색/통찰/시각화분석|- 생략|
|5|데이터모델링(머신러닝모델링)|- 분류 알고리즘 사용<br>- 알고리즘 선택->훈련용 데이터와 학습용 데이터 나눔->학습->예측->평가|
|6|시스템통합|- 생략|

### 2. 데이터 획득/수집

- 모듈 준비

In [ ]:
import urllib.request as req
from bs4 import BeautifulSoup
import os, os.path, gzip
import struct 

- web scraping

In [ ]:
rootUrl = 'http://yann.lecun.com/exdb/mnist/'
soup = BeautifulSoup(req.urlopen(rootUrl),'html5lib')

- 4개의 url 획득

In [ ]:
#모든 요소 tt중에 상위 4개가 링크
for tt in soup.findAll('tt')[:4]:
    print(tt.a.string)
    
files=[tt.a.string for tt in soup.findAll('tt')[:4]]
files

다운로드>압축해제 <- 반복작업

In [ ]:
savePath = './data/mnist'
if not os.path.exists(savePath):
    os.makedirs(savePath)
    
#tqdm:진행율을 보여주는 모듈
from tqdm import tqdm_notebook
for file in tqdm_notebook(files):
    print('소스',rootUrl + file)
    local_path='%s/%s' % (savePath,file)
    print('대상',local_path)
    
    #웹상에 존재하는 리소스를 로컬 디스크상에 직접 저장
    req.urlretrieve(rootUrl + file,local_path)

In [ ]:
#압축해제
for file in tqdm_notebook(files):
    #원본파일의 경로
    ori_gzip_file='%s/%s' % (savePath,file)
    print(ori_gzip_file)
    #압축해제 파일의 경로
    target_file='%s/%s' % (savePath,file[:-3])
    print(target_file)
    #압축해제
    #gzip의 파일오픈 -> 읽기 -> 쓰기
    with gzip.open(ori_gzip_file,'rb') as fg:
        #읽기(압축해제를 수행했다)
        tmp = fg.read()
        #쓰기:일반파일로 기록
        with open(target_file,'wb') as f:
            f.write(tmp)

### 3. 데이터준비/전처리

- 디코딩(내부구조를 알수 있는 인코딩 문서(MNIST Database) 필요)
- 에디언(Endian)처리(TCP/IP상에서 통신 수행시 중요)
    - 컴퓨터 메모리와 같은 1차원 공간에 여러개의 연속된 데이터를 배열하는 방법
    - 종류:바이트를 배치하는 오더(순서)를 앞에서부터 혹은 뒤에서부터 채우는가
        - 0x12345678
        - 빅 에디언 : 값을 앞에서부터 채운다
            0x12 0x34 0x56 0x78
        - 리틀 에디언 : 값을 뒤에서부터 채운다
            0x78 0x56 0x34 0x12
        - 위의 예는 정수값(4byte)을 예를 든것이고, 단지 값이 어떻게 기록됐는지만 이해하고, 그대로 값을 복원할 수 있으면 끝
- 벡터화 처리
- label file
    - magic number : 4byte -> 에디안 체크
    - label 수 : 4byte -> 에디안
    - label 데이터 : 1byte -> 0~9 값
    - 크기 = 4 + 4 + label수*1byte = 8 + 60000 = 60008byte
- image file
    - magic number : 4byte -> 에디안 체크
    - 손글시 이미지 개수 : 4byte -> 에디안 체크
    - 가로크기(픽셀수) : 4byte -> 에디안 체크
    - 세로크기(픽셀수 : 4byte -> 에디안 체크
    - 픽셀값 한개 한개 : unsigned 1byte(=8bit)(0~2^8-1 : 0~255(0xFF))

In [ ]:


def decoding_mnist_rawData(dataStyle='train',maxCount=0):
    label_f=open(f'./data/mnist/{dataStyle}-labels-idx1-ubyte','rb')
    image_f=open(f'./data/mnist/{dataStyle}-images-idx3-ubyte','rb')
    csv_f=open(f'./data/mnist/{dataStyle}.csv','w',encoding='utf-8')
    magic_number, label_count = struct.unpack('>II',label_f.read(8))
    magic_number2, image_count, row, col = struct.unpack('>IIII',image_f.read(16))

    if maxCount>image_count:
        print('개수의 범위를 넘었습니다. 최소 %s개 이내' % image_count)
        return
    elif maxCount== -1:
        maxCount = image_count
    elif maxCount <-1:
        print('개수의 범위를 넘었습니다. 최소 %s개 이내' % image_count)
        return
    
    pixels=row*col 
    for idx in (range(maxCount)):
#         if idx >= maxCount : break
        label_tmp = struct.unpack('B', label_f.read(1))
        label=label_tmp[0]
        binarryData = image_f.read(pixels)
        strData = list(map(lambda x:str(x), binarryData))
        csv_f.write(str(label)+',')
        csv_f.write(','.join(strData) + '\n')

    image_f.close()
    label_f.close()
    csv_f.close()
    


In [2]:
decoding_mnist_rawData(dataStyle='train',maxCount=30000)
decoding_mnist_rawData(dataStyle='t10k',maxCount=10000)

#### [M1] 데이터 품질 향상

- 정확도를 96%목표로 머신러닝 모델을 개선

    [사전조치]
    - 머신러닝 모델을 이용하여 예측시 정확도가 떨어지면 데이터의 품질, 양을 검토한다
    - 양을 점차적으로 늘린다
        - 데이터의 개수를 늘리거나, 비율을 조정(훈련:테스트=75:25)
    - 품질을 향상시킨다
        - 정규화
        - 차후에 적용가능한 내용 : PCA같은 비지도 학습의 차원축소(피처의 수를 줄인다)
        
    [모델개선조치]
    - 알고리즘 교체
    - 하이퍼파라미터 튜닝
    - 파이프라인을 이용한 전처리기를 활용(품질향상)하여 향상
    - 이런 교차 검증법을 활용하여 성능향상을 도모한다
    - 이런 것들의 검증은 ROC곡선, AUC값 등으로 확인할 수도 있고, 교차 검증법의 결과로 확인 가능

In [3]:
def load_csv_ex(dataType='train'):
    labels=list()
    images=list()
    with open(f'./data/mnist/{dataType}.csv','r') as f:
        for line in f:
            tmp=line.split(',')
            labels.append(int(tmp[0]))
            images.append(list(map(lambda x:int(x)/,tmp[1:])))
    return {'labels':labels,'images':images}

In [12]:
train=load_csv_ex()
test=load_csv_ex('t10k')

In [ ]:
### 4	데이터탐색/통찰/시각화분석

- skip

### 5	데이터모델링(머신러닝모델링)

- 지도학습 데이터이므로, 정확도를 통해서 평가를 1차로 수행

In [16]:
train['labels'][:5],train['images'][0][:5]

([5, 0, 4, 1, 9], [0, 0, 0, 0, 0])

In [5]:
from sklearn import svm, model_selection, metrics

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std_train=scaler.fit_transform(train['images'])

In [18]:
clf=svm.SVC()

In [19]:
len(train['images']),len(train['labels'])

(30000, 30000)

In [20]:
clf.fit(X_std_train, train['labels'])

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [21]:
predict=clf.predict(scaler.transform(test['images']))

In [22]:
metrics.accuracy_score(test['labels'],predict)

0.9582

In [23]:
clf_report=metrics.classification_report(test['labels'],predict)
print(clf_report)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       980
           1       0.98      0.99      0.99      1135
           2       0.95      0.96      0.95      1032
           3       0.95      0.96      0.96      1010
           4       0.96      0.96      0.96       982
           5       0.96      0.94      0.95       892
           6       0.97      0.97      0.97       958
           7       0.92      0.95      0.94      1028
           8       0.95      0.94      0.95       974
           9       0.97      0.93      0.95      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000

